# Episodic Memory를 사용한 멀티 에이전트 헬스케어 시스템

## 소개

이 노트북은 AgentCore Memory SDK와 Strands memory hooks를 사용하여 **episodic memory를 갖춘 멀티 에이전트 헬스케어 시스템**을 구현하는 방법을 보여줍니다. 이 접근 방식은 수동 API 호출 없이 자동 메모리 관리를 제공합니다.

### 튜토리얼 세부사항

| 정보         | 세부사항                                                                          |
|:--------------------|:----------------------------------------------------------------------------------|
| 튜토리얼 유형       | 멀티 에이전트 조정을 사용한 Episodic Memory                                    |
| Agent 유형          | 헬스케어 어시스턴트 시스템                                                      |
| Agentic Framework   | Memory Hooks를 사용한 Strands Agents                                                 |
| LLM model           | Anthropic Claude Sonnet 4                                                        |
| 튜토리얼 구성요소 | Episodic Memory, Memory Hooks, HealthLake Integration                           |
| 예제 복잡도  | 중급                                                                     |

학습 내용:

- episodic memory를 위한 MemoryClient SDK 사용 방법
- 자동 메모리 관리를 위한 memory hooks 생성
- 공유 episodic memory를 사용하는 전문 agents 구현
- 실시간 HealthLake FHIR 쿼리 통합

## Episodic Memory가 이 헬스케어 어시스턴트를 돕는 방법

**EpisodicStrategy**는 상호작용을 구조화된 에피소드로 캡처하고 세션 전반에 걸쳐 의미 있는 인사이트를 생성합니다. 이는 "무엇이 일어났는지"를 기록하는 것을 넘어 상호작용이 "왜" 그리고 "어떻게" 전개되었는지를 이해합니다.

### 3단계 프로세스

1. **Extraction** – 단기 메모리(이벤트)에서 유용한 인사이트를 식별하고 구조화된 에피소드로 장기 메모리에 배치
2. **Consolidation** – 정보를 새 에피소드에 작성할지 기존 에피소드를 업데이트할지 결정
3. **Reflection** – 여러 에피소드에 걸쳐 인사이트를 생성하여 패턴과 개선사항 식별

### 에피소드 구조

각 에피소드는 다음을 캡처합니다:
- **Situation**: 헬스케어 전문가가 달성하려고 했던 것
- **Intent**: 상호작용의 주요 목표
- **Assessment**: 목표가 성공적으로 달성되었는지 여부
- **Justification**: 평가가 이루어진 이유
- **Turn-by-turn analysis**: agent 라우팅, tool 사용 및 의사결정을 보여주는 상세 분석
- **Episode-level reflection**: 이 특정 세션에서 잘 작동한 것에 대한 인사이트

### 환자 수준 Reflections

Reflections는 여러 에피소드에 걸쳐 통합되어 더 광범위한 인사이트를 추출합니다:
- **Successful strategies**: 일관되게 작동하는 패턴 (예: 라우팅 프로토콜, 데이터 표시)
- **Common use cases**: 이 환자에 대해 자주 이루어지는 문의 유형
- **Potential improvements**: 어시스턴트가 더 효과적일 수 있는 영역
- **Lessons learned**: 여러 상호작용에 걸친 인사이트

### 헬스케어 워크플로우의 이점

1. **Improved routing**: 어떤 agent가 어떤 유형의 질문을 가장 효과적으로 처리하는지 학습
2. **Better data presentation**: 복잡한 헬스케어 데이터를 빠르게 이해할 수 있도록 포맷하는 방법 이해
3. **Pattern recognition**: 특정 환자에 대한 일반적인 문의 패턴 식별
4. **Quality improvement**: 여러 세션에 걸쳐 무엇이 작동하고 작동하지 않는지 추적
5. **Contextual awareness**: 과거 세션에서 학습한 교훈으로부터 향후 상호작용이 이익을 얻음

이 튜토리얼에서는 에피소드가 멀티 에이전트 상호작용의 전체 흐름을 캡처하는 방법과 reflections가 시간이 지남에 따라 헬스케어 어시스턴트를 개선하기 위한 실행 가능한 인사이트를 제공하는 방법을 확인할 수 있습니다.

---
## 시나리오 컨텍스트

다음과 같은 **헬스케어 어시스턴트 시스템**을 생성합니다:
1. 환자 질문을 라우팅하는 **Supervisor Agent**
2. 보험 및 청구를 위한 **Claims Agent**
3. 환자 정보를 위한 **Demographics Agent**
4. 처방전을 위한 **Medication Agent**

모든 agents는 memory hooks를 사용하여 대화를 episodic memory에 자동으로 저장합니다.

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="75%" />
</div>

## 사전 요구사항

- Python 3.10+
- Bedrock 및 AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon HealthLake datastore (선택사항)

시작해봅시다!

## Step 1: 환경 설정
이 노트북을 작동시키기 위해 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작하겠습니다.

In [ ]:
%pip install -qr ./requirements.txt

In [ ]:
import logging
from datetime import datetime
from botocore.exceptions import ClientError
from strands import Agent, tool  # Strands agent framework
from strands.hooks import HookProvider, HookRegistry  # Memory hook 시스템
from bedrock_agentcore.memory import MemoryClient  # AgentCore Memory SDK

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("healthcare-assistant")

Memory 구성을 위한 사용자 입력을 정의하겠습니다

In [ ]:
MEMORY_NAME = "healthcare_episodic_memory"
PATIENT_ID = "b2055b4d-ac17-4d94-8c5b-3395e4c334dd"  # Synthea 샘플 환자 ID
region = "us-east-1"  # Replace with your AWS region
SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 세션별 고유 ID 생성
MODEL_ID = (
    "global.anthropic.claude-sonnet-4-20250514-v1:0"  # Replace with your Model ID
)

print("Memory Configuration:")
print(f"  Memory Name: {MEMORY_NAME}")
print(f"  Patient ID: {PATIENT_ID}")
print(f"  Region: {region}")
print(f"  Session ID: {SESSION_ID}")
print(f"  Model ID: {MODEL_ID}")

## Step 2: HealthLake Datastore 구성

헬스케어 agents가 쿼리할 환자 데이터로 HealthLake FHIR datastore를 설정합니다.

In [ ]:
import boto3
import requests
import time
from botocore.auth import SigV4Auth  # AWS Signature V4 인증
from botocore.awsrequest import AWSRequest

# HealthLake configuration
HEALTHLAKE_REGION = (
    input("Enter HealthLake region (or press Enter for us-east-1): ").strip()
    or "us-east-1"
)
DATASTORE_ID = input(
    "Enter HealthLake datastore ID (or press Enter to create new): "
).strip()

healthlake_client = boto3.client("healthlake", region_name=HEALTHLAKE_REGION)

# Create new datastore if not provided
if not DATASTORE_ID:
    create_new = (
        input(
            "\nNo datastore ID provided. Create new HealthLake datastore with Synthea data? (yes/no): "
        )
        .strip()
        .lower()
    )

    if create_new == "yes":
        print("\nCreating HealthLake datastore...")

        # Create datastore
        create_response = healthlake_client.create_fhir_datastore(
            DatastoreName=f"healthcare-demo-{int(time.time())}",
            DatastoreTypeVersion="R4",  # FHIR R4 버전 사용
            PreloadDataConfig={"PreloadDataType": "SYNTHEA"},  # Synthea 샘플 데이터 로드
        )

        DATASTORE_ID = create_response["DatastoreId"]
        print(f"✅ Datastore created: {DATASTORE_ID}")
        print(
            "⏳ Waiting for datastore to become ACTIVE (this may take 10-15 minutes)..."
        )

        # Wait for ACTIVE status
        while True:
            status_response = healthlake_client.describe_fhir_datastore(
                DatastoreId=DATASTORE_ID
            )
            status = status_response["DatastoreProperties"]["DatastoreStatus"]

            if status == "ACTIVE":
                print("✅ Datastore is ACTIVE")
                break
            elif status in ["FAILED", "DELETING"]:
                print(f"❌ Datastore creation failed with status: {status}")
                raise Exception(f"Datastore creation failed: {status}")

            print(f"   Status: {status}...")
            time.sleep(30)  # 30초마다 상태 확인

        print(f"\n✅ Synthea data loaded. Using default patient ID: {PATIENT_ID}")


# Get HealthLake endpoint
datastore = healthlake_client.describe_fhir_datastore(DatastoreId=DATASTORE_ID)
HEALTHLAKE_ENDPOINT = datastore["DatastoreProperties"]["DatastoreEndpoint"]


def query_healthlake(resource_type, search_params=None, resource_id=None):
    """Query HealthLake FHIR API"""
    if resource_id:
        url = f"{HEALTHLAKE_ENDPOINT}/{resource_type}/{resource_id}"
    else:
        url = f"{HEALTHLAKE_ENDPOINT}/{resource_type}"
        if search_params:
            params = "&".join([f"{k}={v}" for k, v in search_params.items()])
            url += f"?{params}"

    session = boto3.Session()
    credentials = session.get_credentials()

    # AWS SigV4 서명으로 요청 생성
    request = AWSRequest(
        method="GET", url=url, headers={"Accept": "application/fhir+json"}
    )
    SigV4Auth(credentials, "healthlake", HEALTHLAKE_REGION).add_auth(request)

    response = requests.get(url, headers=dict(request.headers))

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to fetch: {response.text}"}


print(f"\n{'=' * 70}")
print("HealthLake Configuration:")
print(f"  Datastore ID: {DATASTORE_ID}")
print(f"  Endpoint:     {HEALTHLAKE_ENDPOINT}")
print(f"  Region:       {HEALTHLAKE_REGION}")
print(f"  Patient ID:   {PATIENT_ID}")
print(f"{'=' * 70}")

## Step 3: Episodic Strategy로 Memory 생성 

여러 브랜치를 지원하는 단일 memory 리소스를 생성합니다 - 각 agent마다 하나씩. 이 공유 memory 리소스는 기반 역할을 하며, 브랜치는 각 agent의 대화에 대해 격리된 컨텍스트를 제공합니다.

Git 저장소처럼 생각하면 됩니다: 하나의 저장소(memory 리소스)에 여러 브랜치(agent 컨텍스트)가 있습니다.

In [ ]:
client = MemoryClient(region_name=region)

# Episodic memory strategy 정의
strategies = [
    {
        "episodicMemoryStrategy": {
            "name": "HealthcareEpisodes",
            "description": "Captures healthcare interactions as episodes",
            "namespaces": ["healthcare/{actorId}/{sessionId}/"],  # 세션별 episode 저장
            "reflectionConfiguration": {"namespaces": ["healthcare/{actorId}/"]},  # 환자별 reflection 저장
        }
    }
]

try:
    memory = client.create_memory_and_wait(
        name=MEMORY_NAME,
        strategies=strategies,
        description="Healthcare system with episodic memory",
        event_expiry_days=7,  # Short-term conversation expires after 7 days
        max_wait=300,  # 최대 5분 대기
        poll_interval=10,  # 10초마다 상태 확인
    )
    memory_id = memory["id"]
    logger.info(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response["Error"]["Code"] == "ValidationException" and "already exists" in str(
        e
    ):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next(
            (m["id"] for m in memories if m["id"].startswith(MEMORY_NAME)), None
        )
        logger.info(f"Memory already exists. Using existing memory: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback

    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 헬스케어 멀티 에이전트 시스템을 위한 Memory Branching 이해

우리가 생성한 memory 리소스는 **branching**을 지원합니다 - 헬스케어 멀티 에이전트 아키텍처에 중요한 기능입니다. 작동 방식은 다음과 같습니다:

**단일 Memory 리소스, 여러 브랜치:**
- 모든 agents는 동일한 `memory_id`와 `session_id`를 공유
- 각 agent는 격리된 컨텍스트를 위해 자체 `branch_name`을 가짐

**헬스케어 멀티 에이전트 시스템의 주요 이점:**

1. **Context Isolation**: 각 agent는 간섭 없이 자체 대화 기록을 유지
   - Claims agent는 보험 및 청구 대화만 확인
   - Demographics agent는 환자 정보 대화만 확인
   - Medication agent는 처방전 관련 대화만 확인
   - Supervisor는 주요 라우팅 및 조정 흐름을 확인

2. **Parallel Execution Safety**: 여러 agents가 동시에 실행 가능
   - agents가 병렬로 실행될 때 메모리 충돌 없음
   - 각 브랜치는 독립적으로 액세스 가능
   - 동시 처리가 필요한 헬스케어 워크플로우에 중요

3. **Clear Audit Trail**: 각 agent의 상호작용을 추적 가능
   - 각 헬스케어 agent가 논의한 내용 검사
   - agent별 문제 디버그
   - 환자 치료 대화의 흐름 이해
   - 규정 준수 및 문서화 요구사항 유지

**헬스케어 브랜치 구조:**
- `main` 브랜치: Supervisor 라우팅 결정
- `claims_agent` 브랜치: 보험 및 청구 대화
- `demographics_agent` 브랜치: 환자 정보 업데이트
- `medication_agent` 브랜치: 처방전 논의

## Step 4: 브랜치 지원을 사용한 Memory Hook Provider 생성

In [ ]:
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from strands.hooks import AgentInitializedEvent, MessageAddedEvent
from bedrock_agentcore.memory import MemorySessionManager


class HealthcareMemoryHooks(HookProvider):
    def __init__(
        self, memory_id: str, region_name: str = None, branch_name: str = "main"
    ):
        """Initialize the hook with a MemorySessionManager.

        Args:
            memory_id: The AgentCore Memory ID
            region_name: AWS region for the memory service
            branch_name: Branch name for this agent's memory (default: "main")
        """
        if region_name is None:
            region_name = region  # Use global region variable

        self.memory_manager = MemorySessionManager(
            memory_id=memory_id, region_name=region_name
        )
        self.memory_id = memory_id
        self.branch_name = branch_name
        self._sessions = {}  # Cache session objects per actor/session combo
        self._branch_initialized = False  # Track if branch has been created

    def _get_or_create_session(self, actor_id: str, session_id: str):
        """Get or create a MemorySession for the given actor/session."""
        key = f"{actor_id}:{session_id}"
        if key not in self._sessions:
            self._sessions[key] = self.memory_manager.create_memory_session(
                actor_id=actor_id, session_id=session_id
            )
        return self._sessions[key]

    def _initialize_branch(self, actor_id: str, session_id: str):
        """Initialize a branch if it doesn't exist and this is not the main branch."""
        if self._branch_initialized or self.branch_name == "main":
            return

        try:
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Check if branch already exists
            branches = memory_session.list_branches()
            branch_exists = any(b.name == self.branch_name for b in branches)

            if not branch_exists:
                # Get the last event from main branch to fork from
                main_events = memory_session.list_events(branch_name="main")
                if not main_events:
                    # Create initial event in main branch
                    memory_session.add_turns(
                        [
                            ConversationalMessage(
                                "Healthcare system initialized", MessageRole.ASSISTANT
                            )
                        ]
                    )
                    main_events = memory_session.list_events(branch_name="main")

                if main_events:
                    last_event = main_events[-1]
                    # main 브랜치에서 새 브랜치 생성 (fork)
                    memory_session.fork_conversation(
                        root_event_id=last_event.eventId,
                        branch_name=self.branch_name,
                        messages=[
                            ConversationalMessage(
                                f"Starting {self.branch_name} healthcare branch",
                                MessageRole.ASSISTANT,
                            )
                        ],
                    )
                    logger.info(f"✅ Created healthcare branch: {self.branch_name}")

            self._branch_initialized = True

        except Exception as e:
            logger.error(
                f"Failed to initialize healthcare branch {self.branch_name}: {e}"
            )

    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when healthcare agent starts"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning(
                    "Missing actor_id or session_id in healthcare agent state"
                )
                return

            # Initialize branch if needed (for non-main branches)
            if self.branch_name != "main":
                self._initialize_branch(actor_id, session_id)

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Get last 5 conversation turns from this branch
            recent_turns = memory_session.get_last_k_turns(
                k=5, branch_name=self.branch_name, include_parent_branches=False
            )

            if recent_turns:
                # Add context to agent's system prompt
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message.get("role", "unknown").lower()
                        text = message.get("content", {}).get("text", "")
                        if text:
                            context_messages.append(f"{role.title()}: {text}")

                if context_messages:
                    context = "\n".join(context_messages[-10:])  # Last 10 messages
                    # 최근 대화 기록을 system prompt에 추가
                    event.agent.system_prompt += (
                        f"\n\nRecent healthcare conversation history:\n{context}\n\n"
                        "Continue the conversation naturally based on this context."
                    )
                    logger.info(
                        f"✅ Loaded healthcare context from branch '{self.branch_name}'"
                    )

        except Exception as e:
            logger.error(f"Failed to load healthcare conversation history: {e}")

    def on_message_added(self, event: MessageAddedEvent):
        """Store healthcare conversation turns in memory on the appropriate branch"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning(
                    "Missing actor_id or session_id in healthcare agent state"
                )
                return

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Get the last message
            messages = event.agent.messages
            if not messages:
                return

            last_message = messages[-1]
            role_str = last_message.get("role", "").upper()
            content_text = last_message.get("content", [{}])[0].get("text", "")

            if not content_text:
                logger.debug("Skipping empty healthcare message")
                return

            # Map role string to MessageRole enum
            role_mapping = {
                "USER": MessageRole.USER,
                "ASSISTANT": MessageRole.ASSISTANT,
                "TOOL": MessageRole.TOOL,
            }
            message_role = role_mapping.get(role_str, MessageRole.USER)

            # Store the message on the appropriate branch
            if self.branch_name == "main":
                # Main branch - just add turns normally
                memory_session.add_turns(
                    messages=[ConversationalMessage(content_text, message_role)]
                )
            else:
                # Non-main branch - need to append to existing branch
                # Initialize branch if it doesn't exist
                if not self._branch_initialized:
                    self._initialize_branch(actor_id, session_id)

                # 해당 브랜치에 메시지 추가
                memory_session.add_turns(
                    messages=[ConversationalMessage(content_text, message_role)],
                    branch={"name": self.branch_name},
                )

            logger.info(f"Memory saved to healthcare branch: {self.branch_name}")

        except Exception as e:
            logger.error(f"Failed to store healthcare message: {e}")

    def get_session(self, actor_id: str, session_id: str):
        """Get the memory session object for direct access."""
        return self._get_or_create_session(actor_id, session_id)

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register healthcare memory hooks with the registry."""
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)


print("✅ Healthcare memory hook provider defined")

## Step 5: Memory Branching을 사용한 멀티 에이전트 헬스케어 아키텍처 생성

이 섹션에서는 **다른 memory 브랜치**를 사용하는 전문 헬스케어 agents를 생성하여 branching 기능을 시연합니다:

### 헬스케어 Branching 전략:
- **Main Branch**: supervisor의 라우팅 결정을 저장하고 기본 대화 스레드 역할
- **claims_agent Branch**: 보험, 청구 및 클레임 대화를 위한 별도 브랜치
- **demographics_agent Branch**: 환자 정보 및 연락처 세부정보를 위한 별도 브랜치
- **medication_agent Branch**: 처방전 및 약물 대화를 위한 별도 브랜치

각 전문 헬스케어 agent는 자체 브랜치에서 작동하며, 처음 사용될 때 주요 대화에서 자동으로 분기됩니다. 이를 통해:

- 다양한 헬스케어 전문 분야에 대한 독립적인 대화 흐름
- 도메인별 의료 컨텍스트 격리
- 주요 supervisor 대화 스레드 보존
- 헬스케어 데이터 분리 요구사항 준수
- 다양한 유형의 환자 상호작용에 대한 명확한 감사 추적

### 헬스케어 Agent 역할:
- **Supervisor Agent**: 환자 질문을 적절한 전문가에게 라우팅
- **Claims Agent**: 보험 청구, 청구 문의 및 보장 질문 처리
- **Demographics Agent**: 환자 인구통계 정보 및 연락처 업데이트 관리
- **Medication Agent**: 처방전 질문, 복용량 정보 및 약물 관리 처리

이 아키텍처는 민감한 헬스케어 대화가 적절하게 격리되면서도 일관된 환자 치료 경험을 유지하도록 보장합니다.

### 브랜치된 Memory로 Agents 생성

다음으로, 다른 memory 브랜치를 사용하는 시스템 프롬프트를 정의하고 agents를 생성합니다. 동일한 `actor_id`와 `session_id`를 사용하지만 다른 `branch_name` 값을 사용하여 격리된 대화 컨텍스트를 생성하는 방법에 주목하세요:

In [ ]:
# System prompt for the healthcare supervisor
SUPERVISOR_PROMPT = """You are a healthcare supervisor agent. Route patient questions to:
    - Claims Agent: for insurance, billing, claims questions
    - Demographics Agent: for personal info, contact details  
    - Medication Agent: for prescriptions, medications, dosage
    
    Respond briefly and indicate which agent you're routing to."""

# System prompt for the claims specialist
CLAIMS_PROMPT = """You handle insurance claims. Use the get_patient_claims tool to fetch 
    current claim data from HealthLake. Answer questions about claims, billing, and coverage."""

# System prompt for the demographics specialist
DEMOGRAPHICS_PROMPT = """You handle patient demographics. Use the get_patient_demographics tool to 
    fetch current patient data from HealthLake. Answer questions about contact details and personal information."""

# System prompt for the medication specialist
MEDICATION_PROMPT = """You handle medications. Use the get_patient_medications tool to fetch 
    current medication data from HealthLake. Answer questions about prescriptions and dosages."""

## Step 6:  헬스케어 Tools 및 Memory Hooks 설정

먼저, 전문 헬스케어 agents를 위한 HealthLake 데이터 tools와 memory hooks를 생성합니다. 각 agent는 특정 브랜치 이름으로 구성된 자체 memory hook을 가집니다:
- Claims agent는 `claims_agent` 브랜치 사용
- Demographics agent는 `demographics_agent` 브랜치 사용  
- Medication agent는 `medication_agent` 브랜치 사용
- Supervisor agent는 `main` 브랜치 사용

이러한 헬스케어 agents가 호출될 때:
1. hook이 브랜치가 존재하는지 확인
2. 존재하지 않으면 주요 대화에서 새 브랜치를 분기
3. agent의 헬스케어 대화가 전용 브랜치에 저장됨
4. 각 agent는 환자 데이터 프라이버시를 위해 격리된 컨텍스트 유지
5. HealthLake 데이터 tools는 실시간 환자 정보 액세스 제공
6. 적절한 데이터 격리를 통해 헬스케어 규정 준수 유지

**HealthLake 데이터 Tools:**
- `get_patient_claims` - 보험 청구 및 청구 정보 검색
- `get_patient_demographics` - 환자 연락처 및 인구통계 데이터 가져오기
- `get_patient_medications` - 현재 처방전 및 약물 데이터 가져오기

**Memory 브랜치 구조:**
- 각 agent는 자체 memory 컨텍스트로 독립적으로 작동
- 환자 대화는 헬스케어 도메인별로 적절하게 격리됨
- Supervisor는 분리를 유지하면서 agents 간 조정


In [ ]:
# Initialize healthcare memory hooks as None
supervisor_hooks = None
claims_hooks = None
demographics_hooks = None
medication_hooks = None

In [ ]:
@tool
def get_patient_claims(patient_id: str = PATIENT_ID) -> dict:
    """Get patient insurance claims from HealthLake"""
    return query_healthlake("Claim", {"patient": patient_id})


@tool
def get_patient_medications(patient_id: str = PATIENT_ID) -> dict:
    """Get patient medications from HealthLake"""
    return query_healthlake("MedicationRequest", {"patient": patient_id})


@tool
def get_patient_demographics(patient_id: str = PATIENT_ID) -> dict:
    """Get patient demographic information from HealthLake"""
    return query_healthlake("Patient", resource_id=patient_id)


# 각 agent별로 독립적인 memory hook 생성
supervisor_hooks = HealthcareMemoryHooks(memory_id, region, "main")  # Supervisor는 main 브랜치 사용
claims_hooks = HealthcareMemoryHooks(memory_id, region, "claims_agent")  # Claims agent 전용 브랜치
demographics_hooks = HealthcareMemoryHooks(memory_id, region, "demographics_agent")  # Demographics agent 전용 브랜치
medication_hooks = HealthcareMemoryHooks(memory_id, region, "medication_agent")  # Medication agent 전용 브랜치

print("✅ HealthLake tools and memory hooks created")

### 헬스케어 Agents 생성

이제 설정한 tools와 memory hooks를 사용하여 헬스케어 agents를 생성합니다:

- **Supervisor Agent**: 질문 라우팅 (main 브랜치)
- **Claims Agent**: 보험 및 청구 (claims_agent 브랜치)  
- **Demographics Agent**: 환자 정보 (demographics_agent 브랜치)
- **Medication Agent**: 처방전 (medication_agent 브랜치)

각 agent는 전문화된 시스템 프롬프트, 관련 tools 및 격리된 대화를 위한 memory hooks를 가집니다.

In [ ]:
# Create specialized healthcare agents with memory branching
supervisor = Agent(
    model=MODEL_ID,
    system_prompt=SUPERVISOR_PROMPT,
    hooks=[supervisor_hooks],  # main 브랜치 사용
    state={"actor_id": PATIENT_ID, "session_id": SESSION_ID},  # 환자 ID와 세션 ID 저장
)

claims_agent = Agent(
    model=MODEL_ID,
    system_prompt=CLAIMS_PROMPT,
    tools=[get_patient_claims],  # HealthLake claims 조회 tool
    hooks=[claims_hooks],  # claims_agent 브랜치 사용
    state={"actor_id": PATIENT_ID, "session_id": SESSION_ID},
)

demographics_agent = Agent(
    model=MODEL_ID,
    system_prompt=DEMOGRAPHICS_PROMPT,
    tools=[get_patient_demographics],  # HealthLake demographics 조회 tool
    hooks=[demographics_hooks],  # demographics_agent 브랜치 사용
    state={"actor_id": PATIENT_ID, "session_id": SESSION_ID},
)

medication_agent = Agent(
    model=MODEL_ID,
    system_prompt=MEDICATION_PROMPT,
    tools=[get_patient_medications],  # HealthLake medications 조회 tool
    hooks=[medication_hooks],  # medication_agent 브랜치 사용
    state={"actor_id": PATIENT_ID, "session_id": SESSION_ID},
)

print("✅ Healthcare agents created with HealthLake tools and memory branching")

#### Episodic Memory를 사용한 헬스케어 멀티 에이전트 시스템이 준비되었습니다!!

## 헬스케어 어시스턴트를 테스트해봅시다.

환자 치료 시나리오로 헬스케어 멀티 에이전트 시스템을 테스트해봅시다:

**시도해볼 샘플 질문:**
- "What's the status of my insurance claims?"
- "Can you update my contact information?"
- "What medications am I currently taking?"
- "Do I have any pending billing issues?"
- "What's my current address on file?"
- "Are there any drug interactions with my prescriptions?"
- "How much do I owe for my recent visit?"
- "Can you tell me about my coverage details?"

In [ ]:
# Interactive chat with healthcare agents
print("Healthcare Assistant - Type 'quit' to exit\n")

while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["quit", "exit", "q"]:
        break

    if not user_input:
        continue

    # Supervisor가 적절한 agent로 라우팅
    routing = str(supervisor(user_input))
    print(f"\nSupervisor: {routing}")

    # Supervisor의 결정에 따라 적절한 agent로 라우팅
    if "claims agent" in routing.lower():
        response = str(claims_agent(user_input))
        print(f"\nClaims Agent: {response}\n")
    elif "demographics agent" in routing.lower():
        response = str(demographics_agent(user_input))
        print(f"\nDemographics Agent: {response}\n")
    elif "medication agent" in routing.lower():
        response = str(medication_agent(user_input))
        print(f"\nMedication Agent: {response}\n")

## 헬스케어 Memory 브랜치 검사

AgentCore Memory Branching의 주요 장점 중 하나는 각 헬스케어 agent의 대화 기록을 독립적으로 검사할 수 있다는 것입니다. 이는 다음에 중요합니다:

**헬스케어 멀티 에이전트 시스템 디버깅:**
- 각 헬스케어 agent가 환자와 논의한 내용을 정확히 확인
- 어떤 agent가 어떤 의료 문의를 처리했는지 식별
- 헬스케어 시스템을 통한 환자 정보 흐름 추적

**헬스케어 Agent 조정 이해:**
- agents가 별도의 의료 컨텍스트를 유지했는지 확인
- 동시 실행 중 환자 데이터 충돌이 발생하지 않았는지 확인
- 헬스케어 agent 상호작용의 타임라인 감사
- 격리된 대화 추적을 통한 HIPAA 규정 준수 보장

**헬스케어별 이점:**
- **Claims Agent**: 모든 보험 및 청구 논의 추적
- **Demographics Agent**: 환자 정보 업데이트 및 변경 모니터링
- **Medication Agent**: 모든 처방전 및 약물 대화 감사
- **Supervisor Agent**: 라우팅 결정 및 환자 분류 검토

환자 상담 중에 생성된 헬스케어 브랜치를 살펴봅시다:

In [ ]:
print("\n=== Viewing Healthcare Memory Branches ===")

if claims_hooks or demographics_hooks or medication_hooks:
    # 모든 hook이 동일한 세션을 가리키므로 아무 hook이나 사용
    hook = (
        claims_hooks
        if claims_hooks
        else (demographics_hooks if demographics_hooks else medication_hooks)
    )
    if hook:
        memory_session = hook.get_session(actor_id=PATIENT_ID, session_id=SESSION_ID)

        # 세션의 모든 브랜치 조회
        branches = memory_session.list_branches()
        print(f"\n📊 Session has {len(branches)} branches total:")
        for branch in branches:
            events = memory_session.list_events(branch_name=branch.name)
            print(f"  - Branch: {branch.name}")
            print(f"    └─ Events: {len(events)}")
            print(f"    └─ Created: {branch.created}")

            # 브랜치의 최근 대화 출력
            if events:
                print("    └─ Recent conversations:")
                for event in events[-100:]:  # Show last 10 events
                    for payload in event.payload:
                        if "conversational" in payload:
                            role = payload["conversational"]["role"]
                            text = payload["conversational"]["content"]["text"]
                            print(f"        {role}: {text[:500]}...")

        print("\n💡 Each branch represents a different agent's memory:")
        print("  • 'main' = Supervisor agent conversations")
        print("  • 'claims_agent' = Claims assistant conversations")
        print("  • 'demographics_agent' = Demographics assistant conversations")
        print("  • 'medication_agent' = Medication assistant conversations")
else:
    print(
        "No memory hooks found. Make sure to run the cell that creates the hooks first."
    )

## 장기 헬스케어 Memory 검증: Episodes 및 Reflections

**EpisodicStrategy**를 사용하여 헬스케어 시스템이 단기 대화를 구조화된 장기 환자 인사이트로 변환한 방법을 살펴봅시다.

### 헬스케어 Episodes
Episodes는 다음과 함께 통합된 환자 상호작용을 캡처합니다:
- **Clinical Context**: 환자 치료 목표 및 결과
- **Agent Coordination**: supervisor 및 전문 agents가 함께 작업한 방법
- **Data Integration**: HealthLake 정보 검색 및 표시

### 환자 Reflections
Reflections는 다음에 대한 에피소드 간 인사이트를 제공합니다:
- **Care Patterns**: 환자 커뮤니케이션 선호도 및 반복되는 요구사항
- **Effective Strategies**: 이 환자에게 가장 효과적인 접근 방식
- **Optimization Opportunities**: 향후 치료 개선 영역

Episodes 및 reflections는 비동기적으로 처리됩니다.

In [ ]:
print("=== HEALTHCARE LONG-TERM MEMORY: EPISODES & REFLECTIONS ===")
actor_id = PATIENT_ID
session_id = SESSION_ID
# 헬스케어 episode 및 reflection을 위한 namespace 정의
episode_namespace = f"healthcare/{actor_id}/{session_id}/"  # 세션별 episode
reflection_namespace = f"healthcare/{actor_id}/"  # 환자별 reflection
print(f"\n📋 Episode namespace: {episode_namespace}")
print(f"🧠 Reflection namespace: {reflection_namespace}")

try:
    print("\n📖 HEALTHCARE EPISODES (Session-specific patient interactions)")
    # 세션별 episode 조회
    episodes = client.retrieve_memories(
        memory_id=memory_id,
        namespace=episode_namespace,
        query="patient healthcare interactions",
        top_k=10,
    )
    print(f"Found {len(episodes)} healthcare episode(s)")

    for i, episode in enumerate(episodes, 1):
        print(f"\n🏥 Healthcare Episode {i}:")
        content = episode.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            # Show more content for healthcare context
            print(f"   {text[:300]}..." if len(text) > 300 else f"   {text}")
        print(f"   Score: {episode.get('score', 'N/A')}")

    if not episodes:
        print("   No episodes found yet. Episodic processing happens asynchronously.")

except Exception as e:
    print(f"❌ Error retrieving healthcare episodes: {e}")

try:
    print("\n🔍 PATIENT REFLECTIONS (Cross-session healthcare insights)")
    # 환자별 reflection 조회 (여러 세션에 걸친 인사이트)
    reflections = client.retrieve_memories(
        memory_id=memory_id,
        namespace=reflection_namespace,
        query="patient care patterns and insights",
        top_k=10,
    )
    print(f"Found {len(reflections)} patient reflection(s)")

    for i, reflection in enumerate(reflections, 1):
        print(f"\n💡 Patient Reflection {i}:")
        content = reflection.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            # Show more content for healthcare insights
            print(f"   {text[:400]}..." if len(text) > 400 else f"   {text}")
        print(f"   Score: {reflection.get('score', 'N/A')}")

    if not reflections:
        print(
            "   No reflections found yet. Reflections are generated after multiple episodes."
        )

except Exception as e:
    print(f"❌ Error retrieving patient reflections: {e}")

print(
    "\n💡 TIP: Use the memory browser for interactive healthcare memory visualization"
)
print("   Episodes show individual patient consultation summaries")
print("   Reflections reveal patterns in patient care and preferences")
print(
    "\n⏱️  NOTE: Episode and reflection generation takes 10-15 minutes after conversations"
)
print("   Check back later if no episodes/reflections appear immediately")

## 요약

### 구축한 것:
1. **Supervisor Agent** - main 브랜치에서 조정
2. **Claims Agent** - claims_agent 브랜치에서 보험 처리
3. **Demographics Agent** - demographics_agent 브랜치에서 환자 정보 관리
4. **Medication Agent** - medication_agent 브랜치에서 약물 처리

### Memory 아키텍처:
- **단기**: 각 agent는 격리된 브랜치를 가짐
- **Episodes**: 세션별로 저장 `healthcare/{actorId}/{sessionId}/`
- **Reflections**: 모든 세션에 걸쳐 공유 `healthcare/{actorId}/`

### 이점:
- ✅ Agents가 서로의 대화를 방해하지 않음
- ✅ 모든 agents가 동일한 세션의 장기 메모리에 기여
- ✅ 학습된 패턴(reflections)이 모든 환자 세션에 걸쳐 공유됨
- ✅ agent별로 완전한 대화 기록 유지

## 정리 (선택사항)

이 튜토리얼에서 생성된 memory 및 IAM 역할을 삭제하려면 이 셀을 실행하세요.

In [ ]:
# import boto3

# print("Cleanup Options:")
# delete_memory = input("Delete memory? (yes/no): ").strip().lower()
# if delete_memory == 'yes':
#   try:
#       print(f"Deleting memory: {memory_id}")
#       client.delete_memory_and_wait(memory_id=memory_id)
#       print("Memory deleted")
#   except Exception as e:
#       print(f"Error deleting memory: {e}")
# else:
#   print(f"Memory preserved: {memory_id}")

# delete_healthlake = input("Delete HealthLake datastore? (yes/no): ").strip().lower()
# if delete_healthlake == 'yes':
#     try:
#         print(f"Deleting HealthLake datastore: {DATASTORE_ID}")
#         healthlake_client.delete_fhir_datastore(DatastoreId=DATASTORE_ID)
#         print("HealthLake datastore deletion initiated")
#     except Exception as e:
#         print(f"Error deleting HealthLake datastore: {e}")
# else:
#     print(f"HealthLake datastore preserved: {DATASTORE_ID}")

# print("Cleanup complete")